# ElementTree XML API

Experimentation with Python's ElementTree API to be able to use it to look at the diagnostic data

In [84]:
import xml.etree.ElementTree as ET

In [85]:
tree = ET.parse('ladvice_min3000/ladvice_min3000_numtopics5/ladvice_min3000_numtopics5_optinterval50/ladvice_diagnostics_min3000_numtopics5_optinterval50.xml')
root = tree.getroot()

In [86]:
root.tag

'model'

In [87]:
for child in root:
    print(child.tag, child.attrib)

topic {'id': '0', 'tokens': '960699.0000', 'document_entropy': '8.5349', 'word-length': '5.2000', 'coherence': '-1869.5797', 'uniform_dist': '3.5594', 'corpus_dist': '0.6506', 'eff_num_words': '481.2395', 'token-doc-diff': '0.0309', 'rank_1_docs': '0.2227', 'allocation_ratio': '0.2451', 'allocation_count': '0.2789', 'exclusivity': '0.5471'}
topic {'id': '1', 'tokens': '1276123.0000', 'document_entropy': '8.7212', 'word-length': '5.4200', 'coherence': '-1588.1932', 'uniform_dist': '3.5622', 'corpus_dist': '0.5469', 'eff_num_words': '632.4838', 'token-doc-diff': '0.0139', 'rank_1_docs': '0.2378', 'allocation_ratio': '0.2508', 'allocation_count': '0.3069', 'exclusivity': '0.6016'}
topic {'id': '2', 'tokens': '1865544.0000', 'document_entropy': '9.1056', 'word-length': '5.4200', 'coherence': '-1619.8216', 'uniform_dist': '3.5159', 'corpus_dist': '0.3941', 'eff_num_words': '675.3960', 'token-doc-diff': '0.0096', 'rank_1_docs': '0.3173', 'allocation_ratio': '0.3172', 'allocation_count': '0.4

In [88]:
print(root[0].tag, root[0].attrib)

topic {'id': '0', 'tokens': '960699.0000', 'document_entropy': '8.5349', 'word-length': '5.2000', 'coherence': '-1869.5797', 'uniform_dist': '3.5594', 'corpus_dist': '0.6506', 'eff_num_words': '481.2395', 'token-doc-diff': '0.0309', 'rank_1_docs': '0.2227', 'allocation_ratio': '0.2451', 'allocation_count': '0.2789', 'exclusivity': '0.5471'}


In [89]:
for child in root[0]:
    print(child.tag, child.attrib)

word {'rank': '1', 'count': '22666', 'prob': '0.02359', 'cumulative': '0.02359', 'docs': '3747', 'word-length': '3.0000', 'coherence': '0.0000', 'uniform_dist': '0.1762', 'corpus_dist': '0.0458', 'token-doc-diff': '0.0064', 'exclusivity': '0.9749'}
word {'rank': '2', 'count': '18562', 'prob': '0.01932', 'cumulative': '0.04291', 'docs': '3059', 'word-length': '9.0000', 'coherence': '-0.8547', 'uniform_dist': '0.1405', 'corpus_dist': '0.0347', 'token-doc-diff': '0.0053', 'exclusivity': '0.8814'}
word {'rank': '3', 'count': '10441', 'prob': '0.01087', 'cumulative': '0.05378', 'docs': '1061', 'word-length': '3.0000', 'coherence': '-2.6794', 'uniform_dist': '0.0728', 'corpus_dist': '0.0213', 'token-doc-diff': '0.0058', 'exclusivity': '0.9792'}
word {'rank': '4', 'count': '6702', 'prob': '0.00698', 'cumulative': '0.06076', 'docs': '1962', 'word-length': '8.0000', 'coherence': '-1.5516', 'uniform_dist': '0.0436', 'corpus_dist': '0.0069', 'token-doc-diff': '0.0002', 'exclusivity': '0.4101'}
wo

In [90]:
print(root[0][0].attrib)
print(root[0][0].attrib['word-length'])

{'rank': '1', 'count': '22666', 'prob': '0.02359', 'cumulative': '0.02359', 'docs': '3747', 'word-length': '3.0000', 'coherence': '0.0000', 'uniform_dist': '0.1762', 'corpus_dist': '0.0458', 'token-doc-diff': '0.0064', 'exclusivity': '0.9749'}
3.0000


In [91]:
len(list(root))

5

# Topic Coherence

I tried to use the u-mass measure of topic coherence provided by MALLET to choose the optimal number of topics and the best value for opt interval. I trained several models on different settings of num-topics and opt-interval, and then made a table with the average coherence of the topics of each model to see which model had the highest average coherence.

In [92]:
from IPython.display import display, Math, Latex
$$c = \sqrt{a^2 + b^2}$$

SyntaxError: invalid syntax (<ipython-input-92-ab0e6be24cc8>, line 1)

In [82]:
#charmin = the minimum number of characters allowed in the input data
#numtopics = the option num-topics on MALLET
#optintverval = the option opt-interval on MALLET
def getTopicTree(charmin, numtopics, optinterval, strip_legal):
    if strip_legal:
        path = 'ladvice_min{cm}_strip_legal/ladvice_min{cm}_numtopics{nt}/ladvice_min{cm}_numtopics{nt}_optinterval{oi}/ladvice_diagnostics_min{cm}_numtopics{nt}_optinterval{oi}.xml'.format(cm = charmin, nt = numtopics, oi = optinterval)
    else:
        path = 'ladvice_min{cm}/ladvice_min{cm}_numtopics{nt}/ladvice_min{cm}_numtopics{nt}_optinterval{oi}/ladvice_diagnostics_min{cm}_numtopics{nt}_optinterval{oi}.xml'.format(cm = charmin, nt = numtopics, oi = optinterval)
    
    try:
        return ET.parse(path)
    except FileNotFoundError:
        return 0

In [94]:
char_min_list = ['2000', '3000']
num_topics_list = ['5','6','7','8','9','10','20','40']#,'60','80','14','18','20','40','60','80','100']
opt_interval_list = ['10','50','100']

coherences = {}
for char_min in char_min_list:
    table = []
    for num_topics in num_topics_list:
        row = []
        for opt_interval in opt_interval_list:
            tree = getTopicTree(char_min, num_topics, opt_interval, True)
            if tree == 0:
                row.append('----')
                continue
            root = tree.getroot()
            coherence_sum = 0
            for child in root:
                coherence_sum = coherence_sum + float(child.attrib['coherence'])
            row.append(coherence_sum/len(list(root)))
        table.append(row)
    coherences[char_min] = table
    
coherences['3000']

[[-7771.8171999999995, -7814.47984, -7878.86156],
 [-7981.034833333334, -8189.9647, -8094.2973833333335],
 [-8346.982314285715, -8299.994271428572, -8236.297271428573],
 [-8528.301800000001, -8547.317249999998, -8528.0237875],
 [-8676.977677777779, -8581.789477777778, -8785.028266666668],
 [-8923.557250000002, -8831.2263, -8914.755570000001],
 [-10068.403845, -10207.737894999998, -9970.474075],
 [-11334.458990000001, -11359.443075, -11254.0549575]]

### Initial observations of the coherence table:

The average coherence increases (comes closer to 0) when the number of topics decreases. I feel like I am making a mistake while calculating the topic coherence, since it seems to so consistently increase when the topics are decreased. I tested this by training models on the extreme end (2-5 topics), and the coherence keeps increasing, even though 2 topics seems unreasonable to me. Until I figure out what the issue is, I will just compare the models at different numbers of topics myself and refer to the average coherence to examine models at different values of opt-interval.

# General Themes

### Observations based on the top 50 words

I looked at the topic keys for models with 5-10 topics with opt-interval at 10, 50, 100

5 topics:
<ul>
    <li>Money (Employment mostly)</li>
    <li>Family (custody, divorce, etc.)</li>
    <li>Crime </li>
    <li>Property </li>
    <li>Insurance/Damages</li>
</ul>

6 topics:
<ul>
    <li>Money (loans, debt, suing, services, payment)</li>
    <li>Family (custody, divorce, etc.)</li>
    <li>Crime</li>
    <li>Employment </li>
    <li>Property </li>
    <li>Traffic Accidents</li>
</ul>

9 topics:
<ul>
    <li>Money (loans, debt, suing, services, payment)</li>
    <li>Family (custody, divorce, etc.)</li>
    <li>Crime (usually includes sexual violence in the top words)</li>
    <li>Employment </li>
    <li>Property </li>
    <li>Renting </li>
    <li>Traffic Accidents</li>
    <li>Health</li>
    <li>Variable new topic: sometimes split off from crime or property, sometimes an unclear topic with words related to school, copyright, and foreign countries</li>
</ul>


Areas of murky grouping:
<ul>
    <li>Traffic Accidents: With 5 topics and opt-interval = 50,100, and 6 topics opt-interval = 50, words related to car accidents are split between a topic about insurance and damages in general, and Crime. Most of the time the topic is specifically about traffic accidents. There is also some overlap between Traffic Accidents with Property (expected since accidents often cause damage to property). Words related to animals sometimes appear as top words in Traffic Accidents, and sometimes the top words of Traffic Accidents have a lot to do with property. </li><br> 
    <li>Health: With 5 topics, words related to healthcare appear under Family. With 6 topics, they appear under Employment, probably because in the US, health insurance is tied to employment. When I increase the number of topics to 7, it sometimes becomes a separate topic, but other times the model decides to split Property to create a separate topic for renting issues. After 8 topics, Health is usually a separate topic, sometimes focused on animals and vet care, sometimes on mental health issues.</li><br> 
    <li>Animals: Usually words relating to animals are under Property, sometimes they are under Traffic Accidents or Healthcare, and sometimes they are spread out. This is probably because animals can be involved in a lot of different situations. There may be issues with pets in leasing agreements, pets causing noise, pets could be going through medical procedures, etc. As for Traffic Accidents, it could be that cars running over pets is common enough for the model to sometimes group them together, or it could just be the result of the overlap between Traffic Accidents and Property. </li><br> 
    <li>School: Words related to school like "students", "school", "class", "college", "university" usually appear under Employment, Crime, or Family. Just like with Animals, I think this is because of how many different situations school could be involved in. People could be getting into academic trouble that could have academic implications like plagerism and cheating (Crime), students could be caught with drugs at school (Crime), separated parents could argue about how to arrange children's schooling (Family), university students could have problems with unemployment or discrimination (Employment). I am less clear on all the ways school is related to employment under the umbrella of legal issues, so I think I will examine that further when looking at models with more topics. <br> Words related to school also sometimes become part of a separate topic, though what this topic is about is unclear. It includes words related to websites, businesses, copyright, visas, foreign countries. I think it is some combination of copyright issues and immigration issues, school being the what causes them to overlap. Increasing to 10 topics does not make this topic clear, but I think after 11 or 12 topics it would split into two topics. </li><br> 
</ul>